In [1]:
import re
from time import sleep
from turtle import right
from requests import request
from bs4 import BeautifulSoup
import requests

from pandas_json import DataFrameOutput

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}
response = requests.get("https://locations.buffalos.com/site-map/all", headers=headers)

soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())
data = []
print(soup.find_all("a", href=True))

ConnectionError: HTTPSConnectionPool(host='locations.buffalos.com', port=443): Max retries exceeded with url: /site-map/all (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001D71A08A240>: Failed to resolve 'locations.buffalos.com' ([Errno 11001] getaddrinfo failed)"))

In [12]:
import requests
import json
import time

# Replace with the actual domain
BASE_DOMAIN = "https://www.pretzelmaker.com/"
BASE_URL = f"{BASE_DOMAIN}/wp-admin/admin-ajax.php"
HOME_URL = BASE_DOMAIN  # URL to send the initial GET request to

# United States bounding box coordinates
MIN_LAT = 24.5  # Southernmost point
MAX_LAT = 49.5  # Northernmost point
MIN_LNG = -124.8  # Westernmost point
MAX_LNG = -66.9  # Easternmost point

# Grid step sizes (in degrees)
LAT_STEP = 1.0
LNG_STEP = 1.0

# Search parameters
MAX_RESULTS = 500
SEARCH_RADIUS = 100  # in miles or kilometers, depending on the API

# Dictionary to store unique stores
collected_stores = {}

# Create a session object
session = requests.Session()
session.headers.update(
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/",
    }
)

# Send a GET request to the home page to get cookies
try:
    print(f"Initializing session by visiting {HOME_URL}")
    response = session.get(HOME_URL)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Failed to initialize session: {e}")
    exit(1)


# Function to make a request to the store locator endpoint
def fetch_stores(lat, lng):
    params = {
        "action": "store_search",
        "lat": lat,
        "lng": lng,
        "max_results": MAX_RESULTS,
        "search_radius": SEARCH_RADIUS,
    }
    try:
        response = session.get(BASE_URL, params=params)
        response.raise_for_status()
        stores = response.json()
        return stores
    except requests.exceptions.RequestException as e:
        print(f"Request failed for lat={lat}, lng={lng}: {e}")
        return []
    except json.JSONDecodeError:
        print(f"Failed to decode JSON for lat={lat}, lng={lng}")
        return []


# Main loop to iterate over the grid
lat = MIN_LAT
while lat <= MAX_LAT:
    lng = MIN_LNG
    while lng <= MAX_LNG:
        print(f"Fetching stores for lat={lat}, lng={lng}")
        stores = fetch_stores(lat, lng)
        for store in stores:
            # Adjust the key extraction based on the API response
            store_id = (
                store.get("id")
                or store.get("store_id")
                or (store.get("store") and store["store"].get("id"))
            )
            if store_id and store_id not in collected_stores:
                collected_stores[store_id] = store
        lng += LNG_STEP
        time.sleep(0.2)  # Sleep to avoid overwhelming the server
    lat += LAT_STEP

# Output the total number of unique stores collected
print(f"Total unique stores collected: {len(collected_stores)}")

# Optionally, write the collected stores to a JSON file
with open("stores.json", "w") as f:
    json.dump(list(collected_stores.values()), f, indent=2)

Initializing session by visiting https://www.pretzelmaker.com/
Fetching stores for lat=24.5, lng=-124.8
Fetching stores for lat=24.5, lng=-123.8
Fetching stores for lat=24.5, lng=-122.8
Fetching stores for lat=24.5, lng=-121.8
Fetching stores for lat=24.5, lng=-120.8
Fetching stores for lat=24.5, lng=-119.8
Fetching stores for lat=24.5, lng=-118.8
Fetching stores for lat=24.5, lng=-117.8
Fetching stores for lat=24.5, lng=-116.8
Fetching stores for lat=24.5, lng=-115.8
Fetching stores for lat=24.5, lng=-114.8
Fetching stores for lat=24.5, lng=-113.8
Fetching stores for lat=24.5, lng=-112.8
Fetching stores for lat=24.5, lng=-111.8
Fetching stores for lat=24.5, lng=-110.8
Fetching stores for lat=24.5, lng=-109.8
Fetching stores for lat=24.5, lng=-108.8
Fetching stores for lat=24.5, lng=-107.8
Fetching stores for lat=24.5, lng=-106.8
Fetching stores for lat=24.5, lng=-105.8
Fetching stores for lat=24.5, lng=-104.8
Fetching stores for lat=24.5, lng=-103.8
Fetching stores for lat=24.5, lng=-

In [14]:
import requests
import json
import time

# Replace with the actual domain
BASE_URL = "https://www.pretzelmaker.com//wp-admin/admin-ajax.php"

# Contiguous United States bounding box coordinates
CONUS_MIN_LAT = 24.5
CONUS_MAX_LAT = 49.5
CONUS_MIN_LNG = -124.8
CONUS_MAX_LNG = -66.9

# Hawaii bounding box coordinates
HAWAII_MIN_LAT = 18.5
HAWAII_MAX_LAT = 22.5
HAWAII_MIN_LNG = -160.5
HAWAII_MAX_LNG = -154.5

# Grid step sizes (in degrees)
LAT_STEP = 1.0
LNG_STEP = 1.0

# Search parameters
MAX_RESULTS = 500
SEARCH_RADIUS = 100  # in miles or kilometers, depending on the API

# Dictionary to store unique stores
collected_stores = {}

# Create a session object
session = requests.Session()
session.headers.update(
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/",
    }
)
# Send a GET request to the home page to get cookies
try:
    print(f"Initializing session by visiting {HOME_URL}")
    response = session.get(HOME_URL)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Failed to initialize session: {e}")
    exit(1)


# Function to make a request to the store locator endpoint
def fetch_stores(lat, lng):
    params = {
        "action": "store_search",
        "lat": lat,
        "lng": lng,
        "max_results": MAX_RESULTS,
        "search_radius": SEARCH_RADIUS,
    }
    try:
        response = session.get(BASE_URL, params=params)
        response.raise_for_status()
        stores = response.json()
        return stores
    except requests.exceptions.RequestException as e:
        print(f"Request failed for lat={lat}, lng={lng}: {e}")
        return []
    except json.JSONDecodeError:
        print(f"Failed to decode JSON for lat={lat}, lng={lng}")
        return []


# Function to scan a region defined by latitude and longitude boundaries
def scan_region(min_lat, max_lat, min_lng, max_lng):
    lat = min_lat
    while lat <= max_lat:
        lng = min_lng
        while lng <= max_lng:
            print(f"Fetching stores for lat={lat}, lng={lng}")
            stores = fetch_stores(lat, lng)
            for store in stores:
                store_id = (
                    store.get("id")
                    or store.get("store_id")
                    or (store.get("store") and store["store"].get("id"))
                )
                if store_id and store_id not in collected_stores:
                    collected_stores[store_id] = store
            lng += LNG_STEP
            time.sleep(0.1)  # Sleep to avoid overwhelming the server
        lat += LAT_STEP


# Scan the contiguous United States
scan_region(CONUS_MIN_LAT, CONUS_MAX_LAT, CONUS_MIN_LNG, CONUS_MAX_LNG)

# Scan Hawaii
scan_region(HAWAII_MIN_LAT, HAWAII_MAX_LAT, HAWAII_MIN_LNG, HAWAII_MAX_LNG)
# Alaska bounding box coordinates
ALASKA_MIN_LAT = 54.0
ALASKA_MAX_LAT = 71.5
ALASKA_MIN_LNG = -168.0
ALASKA_MAX_LNG = -140.0

# Scan Alaska
scan_region(ALASKA_MIN_LAT, ALASKA_MAX_LAT, ALASKA_MIN_LNG, ALASKA_MAX_LNG)

# Output the total number of unique stores collected
print(f"Total unique stores collected: {len(collected_stores)}")

# Optionally, write the collected stores to a JSON file
with open("stores.json", "w") as f:
    json.dump(list(collected_stores.values()), f, indent=2)

Initializing session by visiting https://www.pretzelmaker.com/
Fetching stores for lat=24.5, lng=-124.8
Fetching stores for lat=24.5, lng=-123.8
Fetching stores for lat=24.5, lng=-122.8
Fetching stores for lat=24.5, lng=-121.8
Fetching stores for lat=24.5, lng=-120.8
Fetching stores for lat=24.5, lng=-119.8
Fetching stores for lat=24.5, lng=-118.8
Fetching stores for lat=24.5, lng=-117.8
Fetching stores for lat=24.5, lng=-116.8
Fetching stores for lat=24.5, lng=-115.8
Fetching stores for lat=24.5, lng=-114.8
Fetching stores for lat=24.5, lng=-113.8
Fetching stores for lat=24.5, lng=-112.8
Fetching stores for lat=24.5, lng=-111.8
Fetching stores for lat=24.5, lng=-110.8
Fetching stores for lat=24.5, lng=-109.8
Fetching stores for lat=24.5, lng=-108.8
Fetching stores for lat=24.5, lng=-107.8
Fetching stores for lat=24.5, lng=-106.8
Fetching stores for lat=24.5, lng=-105.8
Fetching stores for lat=24.5, lng=-104.8
Fetching stores for lat=24.5, lng=-103.8
Fetching stores for lat=24.5, lng=-

"https://ordernew-admin.greatamericancookies.com/api/v1/restaurants/nearby?delivery=pickup&lat=39.9597&lng=-75.1995"
[
    {
        "id": 124081,
        "name": "Whiteland Towne Center (GAC+MSC)",
        "slug": "great-american-cookies-whiteland-towne-center",
        "location_slug": "137-w-lincoln-hwy-exton-pa-19341",
        "telephone": "(484) 872-8236",
        "cobranded": true,
        "status": {
            "available": true,
            "open": true,
            "message": null,
            "message_custom": "Online Cookie Cake orders require a 2-hour minimum lead time. Please keep in mind that some custom designs may be restricted by copyright laws, and we do not produce any cakes that use profanity or vulgar messages. If you need your Cookie Cake earlier or don't see the design you want, please call the store directly. We are happy to help!",
        },
        "payment": {
            "split": false,
            "tip": true,
            "coupon": true,
            "cards": "American Express\/Discover\/MasterCard\/Visa",
            "loyalty": true,
        },
        "services": {
            "asap": true,
            "advance": true,
            "advance_days": 30,
            "curbside": false,
            "delivery": false,
            "dispatch": true,
            "pickup": true,
        },
        "address": {
            "street": "137 W. Lincoln Hwy",
            "city": "Exton",
            "state": "PA",
            "zip": "19341",
            "country": "US",
            "latitude": 40.029754,
            "longitude": -75.63208,
            "cross_street": "",
            "directions": "https:\/\/maps.google.com\/maps?q=40.029754,-75.63208",
            "utcoffset": -4,
            "distance": 23.41,
        },
        "data": {"attributes": ["CNC"], "customfields": null, "metadata": null},
        "hours": {
            "business": {
                "Mon": {
                    "start": "2024-10-28 12:00:00",
                    "end": "2024-10-28 21:00:00",
                    "is_open": true,
                },
                "Tue": {
                    "start": "2024-10-29 12:00:00",
                    "end": "2024-10-29 21:00:00",
                    "is_open": false,
                },
                "Wed": {
                    "start": "2024-10-30 12:00:00",
                    "end": "2024-10-30 21:00:00",
                    "is_open": false,
                },
                "Thu": {
                    "start": "2024-10-31 12:00:00",
                    "end": "2024-10-31 21:00:00",
                    "is_open": false,
                },
                "Fri": {
                    "start": "2024-11-01 12:00:00",
                    "end": "2024-11-01 22:00:00",
                    "is_open": false,
                },
                "Sat": {
                    "start": "2024-11-02 11:00:00",
                    "end": "2024-11-02 22:00:00",
                    "is_open": false,
                },
                "Sun": {
                    "start": "2024-11-03 11:00:00",
                    "end": "2024-11-03 21:00:00",
                    "is_open": false,
                },
            },
            "dispatch": {
                "Mon": {
                    "start": "2024-10-28 12:00:00",
                    "end": "2024-10-28 20:00:00",
                    "is_open": true,
                },
                "Tue": {
                    "start": "2024-10-29 12:00:00",
                    "end": "2024-10-29 20:00:00",
                    "is_open": false,
                },
                "Wed": {
                    "start": "2024-10-30 12:00:00",
                    "end": "2024-10-30 20:00:00",
                    "is_open": false,
                },
                "Thu": {
                    "start": "2024-10-31 12:00:00",
                    "end": "2024-10-31 20:00:00",
                    "is_open": false,
                },
                "Fri": {
                    "start": "2024-11-01 12:00:00",
                    "end": "2024-11-01 21:00:00",
                    "is_open": false,
                },
                "Sat": {
                    "start": "2024-11-02 11:00:00",
                    "end": "2024-11-02 21:00:00",
                    "is_open": false,
                },
                "Sun": {
                    "start": "2024-11-03 11:00:00",
                    "end": "2024-11-03 20:00:00",
                    "is_open": false,
                },
            },
            "rails": {
                "Mon": {
                    "start": "2024-10-28 12:00:00",
                    "end": "2024-10-28 20:00:00",
                    "is_open": true,
                },
                "Tue": {
                    "start": "2024-10-29 12:00:00",
                    "end": "2024-10-29 20:00:00",
                    "is_open": false,
                },
                "Wed": {
                    "start": "2024-10-30 12:00:00",
                    "end": "2024-10-30 20:00:00",
                    "is_open": false,
                },
                "Thu": {
                    "start": "2024-10-31 12:00:00",
                    "end": "2024-10-31 20:00:00",
                    "is_open": false,
                },
                "Fri": {
                    "start": "2024-11-01 12:00:00",
                    "end": "2024-11-01 21:00:00",
                    "is_open": false,
                },
                "Sat": {
                    "start": "2024-11-02 11:00:00",
                    "end": "2024-11-02 21:00:00",
                    "is_open": false,
                },
                "Sun": {
                    "start": "2024-11-03 11:00:00",
                    "end": "2024-11-03 20:00:00",
                    "is_open": false,
                },
            },
        },
    }
]

In [16]:
import requests
import json
import time

# Replace with the actual domain
BASE_URL = "https://ordernew-admin.greatamericancookies.com/api/v1/restaurants/nearby"
HOME_URL = "https://www.marbleslab.com/"
# Contiguous United States bounding box coordinates
CONUS_MIN_LAT = 24.5
CONUS_MAX_LAT = 49.5
CONUS_MIN_LNG = -124.8
CONUS_MAX_LNG = -66.9

# Hawaii bounding box coordinates
HAWAII_MIN_LAT = 18.5
HAWAII_MAX_LAT = 22.5
HAWAII_MIN_LNG = -160.5
HAWAII_MAX_LNG = -154.5

# Grid step sizes (in degrees)
LAT_STEP = 1.0
LNG_STEP = 1.0

# Search parameters
MAX_RESULTS = 500
SEARCH_RADIUS = 100  # in miles or kilometers, depending on the API
DELIVERY_TYPE = "pickup"
# Dictionary to store unique stores
collected_stores = {}

# Create a session object
session = requests.Session()
session.headers.update(
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/",
    }
)
# Send a GET request to the home page to get cookies
try:
    print(f"Initializing session by visiting {HOME_URL}")
    response = session.get(HOME_URL)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Failed to initialize session: {e}")
    exit(1)


# Function to make a request to the store locator endpoint
def fetch_stores(lat, lng):
    params = {"delivery": DELIVERY_TYPE, "lat": lat, "lng": lng}
    try:
        response = session.get(BASE_URL, params=params)
        response.raise_for_status()
        stores = response.json()
        return stores
    except requests.exceptions.RequestException as e:
        print(f"Request failed for lat={lat}, lng={lng}: {e}")
        return []
    except json.JSONDecodeError:
        print(f"Failed to decode JSON for lat={lat}, lng={lng}")
        return []


# Function to scan a region defined by latitude and longitude boundaries
def scan_region(min_lat, max_lat, min_lng, max_lng):
    lat = min_lat
    while lat <= max_lat:
        lng = min_lng
        while lng <= max_lng:
            print(f"Fetching stores for lat={lat}, lng={lng}")
            stores = fetch_stores(lat, lng)
            for store in stores:
                store_id = (
                    store.get("id")
                    or store.get("store_id")
                    or (store.get("store") and store["store"].get("id"))
                )
                if store_id and store_id not in collected_stores:
                    collected_stores[store_id] = store
            lng += LNG_STEP
            time.sleep(0.1)  # Sleep to avoid overwhelming the server
        lat += LAT_STEP


# Scan the contiguous United States
scan_region(CONUS_MIN_LAT, CONUS_MAX_LAT, CONUS_MIN_LNG, CONUS_MAX_LNG)

# Scan Hawaii
scan_region(HAWAII_MIN_LAT, HAWAII_MAX_LAT, HAWAII_MIN_LNG, HAWAII_MAX_LNG)
# Alaska bounding box coordinates
ALASKA_MIN_LAT = 54.0
ALASKA_MAX_LAT = 71.5
ALASKA_MIN_LNG = -168.0
ALASKA_MAX_LNG = -140.0

# Scan Alaska
scan_region(ALASKA_MIN_LAT, ALASKA_MAX_LAT, ALASKA_MIN_LNG, ALASKA_MAX_LNG)

# Output the total number of unique stores collected
print(f"Total unique stores collected: {len(collected_stores)}")

# Optionally, write the collected stores to a JSON file
with open("stores2.json", "w") as f:
    json.dump(list(collected_stores.values()), f, indent=2)

Initializing session by visiting https://www.marbleslab.com/
Fetching stores for lat=24.5, lng=-124.8
Request failed for lat=24.5, lng=-124.8: 500 Server Error: Internal Server Error for url: https://ordernew-admin.greatamericancookies.com/api/v1/restaurants/nearby?delivery=pickup&lat=24.5&lng=-124.8
Fetching stores for lat=24.5, lng=-123.8
Request failed for lat=24.5, lng=-123.8: 500 Server Error: Internal Server Error for url: https://ordernew-admin.greatamericancookies.com/api/v1/restaurants/nearby?delivery=pickup&lat=24.5&lng=-123.8
Fetching stores for lat=24.5, lng=-122.8
Request failed for lat=24.5, lng=-122.8: 500 Server Error: Internal Server Error for url: https://ordernew-admin.greatamericancookies.com/api/v1/restaurants/nearby?delivery=pickup&lat=24.5&lng=-122.8
Fetching stores for lat=24.5, lng=-121.8
Request failed for lat=24.5, lng=-121.8: 500 Server Error: Internal Server Error for url: https://ordernew-admin.greatamericancookies.com/api/v1/restaurants/nearby?delivery=pi

KeyboardInterrupt: 